In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7,7)
import tqdm

In [9]:
df = pd.read_csv('df.csv')

# These fix formatting issues.
df.drop('Unnamed: 0', axis=1, inplace=True)
df['starttime'] = df['starttime'].apply(lambda x: x[1:-1].split(','))
df['month'] = df['starttime'].apply(lambda x: int(x[1]))
df['date'] = pd.to_datetime(df['starttime'].apply(lambda x: ' -'.join(x[:3])))

### Data Cleaning

In [11]:
# around 60k people rerack their bike at the same station. Doesn't help with our analysis of routes. 
df = df[df['station_start'] != df['station_end']]
#  Create a new column that can be used for convenience.
df['route'] = df['start_addy'] + ' ' + df['addy_end']

In [5]:
df['route'].nunique()

12036

In [12]:
# Isolate our analysis to the top 500 most popular routes to minimize noise and maximize signal.
top_500_df = df[df['route'].isin(df['route'].value_counts()[:500].index)]

starttime  duration  bikenum  station_start  \
0    [2010,  9,  20,  11,  27,  4]      1012    742.0          31208   
2    [2010,  9,  20,  12,  5,  37]      2690    993.0          31600   
4   [2010,  9,  20,  12,  10,  43]      1413    883.0          31100   
5   [2010,  9,  20,  12,  14,  27]       982    850.0          31109   
15  [2010,  9,  20,  12,  39,  35]       351    685.0          31103   

                       start_addy                              addy_end  \
0        M St & New Jersey Ave SE                         4th & M St SW   
2                   5th & K St NW         19th St & Pennsylvania Ave NW   
4   19th St & Pennsylvania Ave NW                        15th & P St NW   
5                   7th & T St NW  Massachusetts Ave & Dupont Circle NW   
15           16th & Harvard St NW           Adams Mill & Columbia Rd NW   

    station_end  member  days_since_Jan1_2010 day_of_week  month       date  \
0         31108    True                   262      Monday      9 2010-09-20   
2         31100    True                   262      Monday      9 2010-09-20   
4         31201    True                   262      Monday      9 2010-09-20   
5         31200    True                   262      Monday      9 2010-09-20   
15        31104    True                   262      Monday      9 2010-09-20   

                                                route  
0              M St & New Jersey Ave SE 4th & M St SW  
2         5th & K St NW 19th St & Pennsylvania Ave NW  
4        19th St & Pennsylvania Ave NW 15th & P St NW  
5   7th & T St NW Massachusetts Ave & Dupont Circl...  
15   16th & Harvard St NW Adams Mill & Columbia Rd NW

In [15]:
df['bikenum'].nunique()
sum(df['bikenum'].isna())

5292

### **Weather data concatenation**

In [13]:
# https://www.visualcrossing.com/weather/weather-data-services

weather = pd.read_csv('Washington 2010-09-20 to 2011-12-31.csv')
weather['datetime'] = pd.to_datetime(weather['datetime'])
weather = weather[['datetime','temp', 'conditions']]

temp = []
conditions = []
for i in range(len(weather)):
    num = sum(df['date'] == weather['datetime'][i])
    array = np.ones(num) * weather['temp'][i]
    cond = [weather['conditions'][i]] * num
    temp.append(array)
    conditions.append(cond)

temp_fin = []
for i in temp:
    temp_fin.extend(i)

temp_cond = []
for i in conditions:
    temp_cond.extend(i)

df['temperature'] = temp_fin
df['conditions'] = temp_cond

In [ ]:
# This removes outliers above or below 2 standard deviations from the mean

length_before_outliers = len(df)
standard_dev = df.groupby('route')['duration'].std() * 2
remove_index = []
outliers_above = dict(df.groupby('route')['duration'].mean() + standard_dev)

for k,v in tqdm(outliers_above.items()):
    route_df = df[df['route'] == k]
    lower = route_df['duration'].mean() - standard_dev[k]
    outlier_index = route_df[route_df['duration'].apply(lambda x: x >= v or x <= lower)].index
    df.drop(outlier_index, inplace=True)

print('number of samples before outliers are removed: ', length_before_outliers)
print('number of samples after outliers are removed:  ', len(df))

top_500_df = df[df['route'].isin(df['route'].value_counts()[:500].index)]
top_500_df.to_csv('final_df.csv')
